<h1> Text Representation </h1>

When we have to deal with text processing, we cannot use word directly to calculate or evaluate on the model. So the first thing we have to do is to encode the text to some numeric representation. The most popular representation for texts is "vector".

Before we go any further, I would like to present you guys for the easiest way to represent the text as a vector, that is "one-hot vector."

In [26]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
vocab = np.array(['animal','bird','cow','dolphin'])
onehot_encoder = OneHotEncoder(categories='auto')
one_hot = onehot_encoder.fit_transform(vocab.reshape(-1,1))
print(one_hot.toarray())
test_doc = np.array(['animal','dolphin'])
test_onehot = onehot_encoder.transform(test_doc.reshape(-1,1)).toarray()
print(test_onehot)
onehot_encoder.inverse_transform(test_onehot)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]]


array([['animal'],
       ['dolphin']], dtype='<U7')